# AutoGraph和tf.Module
这里使用tf.Module来更好地构建Autograph

## 一，Autograph和tf.Module概述


前面在介绍Autograph的编码规范时提到构建Autograph时应该避免在@tf.function修饰的函数内部定义tf.Variable. 

但是如果在函数外部定义tf.Variable的话，又会显得这个函数有外部变量依赖，封装不够完美。

一种简单的思路是定义一个类，并将相关的tf.Variable创建放在类的初始化方法中。而将函数的逻辑放在其他方法中。

但是封装到普通类里面又不太自然。

惊喜的是，TensorFlow提供了一个基类tf.Module，通过继承它构建子类，我们不仅可以获得以上的自然而然，而且可以非常方便地管理变量，还可以非常方便地管理它引用的其它Module，最重要的是，我们能够利用tf.saved_model保存模型并实现跨平台部署使用。

实际上，tf.keras.models.Model,tf.keras.layers.Layer 都是继承自tf.Module的，提供了方便的变量管理和所引用的子模块管理的功能。

**因此，利用tf.Module提供的封装，再结合TensoFlow丰富的低阶API，实际上我们能够基于TensorFlow开发任意机器学习模型(而非仅仅是神经网络模型)，并实现跨平台部署使用。**


## 二、应用tf.Module封装Autograph

In [2]:
import tensorflow as tf

In [22]:
tf.keras.backend.clear_session()
x = tf.Variable(1.0, dtype = tf.float32)

# 使用input_sigature限定输入的参数类型
@tf.function(input_signature = [tf.TensorSpec(shape = [], dtype = tf.float32)])
def add_print(a):
    x.assign_add(a)
    tf.print(x)
    return x

In [23]:
add_print(tf.constant(3.0))

4


<tf.Tensor: shape=(), dtype=float32, numpy=4.0>

In [24]:
add_print(tf.constant(3))  # 不符合签名就会报错

ValueError: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(3, shape=(), dtype=int32))
  input_signature: (
    TensorSpec(shape=(), dtype=tf.float32, name=None))

下面利用tf.Module的子类将其封装一下

In [57]:
class DemoModule(tf.Module):
    def __init__(self, init_value = tf.constant(0.0), name = None):
        # print(type(self).__name__)
        # help(super)
        super(DemoModule, self).__init__(name = name)  # 调用父类的__init__函数 
        with self.name_scope:
            self.x = tf.Variable(init_value, dtype = tf.float32, trainable = True)
            
    @tf.function(input_signature = [tf.TensorSpec(shape = [], dtype = tf.float32)])
    def add_print(self, a):
        with self.name_scope:
            self.x.assign_add(a)
            tf.print(self.x)
            return self.x

In [58]:
print(type(DemoModule))  # type 类型
demo = DemoModule(init_value = tf.constant(1.0))
result = demo.add_print(tf.constant(5.0))
print(demo.variables)
print(demo.trainable_variables)

<class 'type'>
6
(<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=6.0>,)
(<tf.Variable 'demo_module/Variable:0' shape=() dtype=float32, numpy=6.0>,)


In [59]:
# 查看模块中的所有子模块
demo.submodules

()

In [60]:
# 保存模型, 并指定跨平台的方法
tf.saved_model.save(demo, "4-5_model", signatures = {"serving_default":demo.add_print})

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: 4-5_model/assets


In [61]:
# 加载模型
demo2 = tf.saved_model.load("4-5_model")
demo2.add_print(tf.constant(5.0))

11


<tf.Tensor: shape=(), dtype=float32, numpy=11.0>

In [62]:
# 查看模型文件相关信息
!saved_model_cli show --dir ./4-5_model --all

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [66]:
logdir = "4-5_model_log"
writer = tf.summary.create_file_writer(logdir)

# 开启跟踪
tf.summary.trace_on(graph = True, profiler = True)\
# 执行autograph
demo = DemoModule(init_value = tf.constant(0.0))
result = demo.add_print(tf.constant(5.0))

# 将计算图信息写入日志
with writer.as_default():
    tf.summary.trace_export(
        name = "demomodule",
        step = 0,
        profiler_outdir = logdir
    )

5


In [68]:
from tensorboard import notebook
notebook.start("--logdir " + logdir)

除了利用tf.Module的子类实现封装, 还可以给tf.Module添加属性进行封装

In [73]:
mymodule = tf.Module()
mymodule.x = tf.Variable(0.0)

@tf.function(input_signature = [tf.TensorSpec(shape = [], dtype = tf.float32)])
def add_print(a):
    mymodule.x.assign_add(a)
    tf.print(mymodule.x)
    return mymodule.x

mymodule.add_print = add_print

In [74]:
mymodule.add_print(tf.constant(1.0))

1


<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

## 三、tf.Module和tf.keras.Module, tf.keras.layers.Layer
keras中的模型都是继承tf.Module而实现的, 所以也具有管理变量和子模块的功能

In [76]:
print(issubclass(tf.keras.Model, tf.Module))

True


In [78]:
from tensorflow.keras import models, layers, losses, metrics
model = models.Sequential()
model.add(layers.Dense(4, input_shape = (10,)))
model.add(layers.Dense(2))
model.add(layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 44        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [79]:
model.variables

[<tf.Variable 'dense/kernel:0' shape=(10, 4) dtype=float32, numpy=
 array([[-0.14765835, -0.44911543,  0.46934175,  0.34641838],
        [-0.62472486,  0.21721846,  0.4384011 ,  0.07518214],
        [ 0.26122022,  0.33657342,  0.28380346,  0.5521077 ],
        [ 0.12154728,  0.4929018 ,  0.2784471 , -0.0681749 ],
        [ 0.48611975, -0.56647366,  0.417377  , -0.04760611],
        [-0.62441283,  0.28494382,  0.28925323, -0.1252951 ],
        [-0.3382162 , -0.05886042,  0.5098765 , -0.24826556],
        [ 0.50220203,  0.155334  , -0.6104601 , -0.58940935],
        [ 0.62473285, -0.03757823,  0.11644346, -0.18094835],
        [-0.6079622 ,  0.46762598,  0.5891422 ,  0.42066717]],
       dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[-0.7403352 ,  0.5320606 ],
        [ 0.9753165 ,  0.47099924],
        [ 0.14637184, -0.5364406 ],
        [ 

In [80]:
model.layers[0].trainable = False  # 冻结第0层，让其不能训练
model.trainable_variables

[<tf.Variable 'dense_1/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[-0.7403352 ,  0.5320606 ],
        [ 0.9753165 ,  0.47099924],
        [ 0.14637184, -0.5364406 ],
        [ 0.88597775, -0.50798845]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'dense_2/kernel:0' shape=(2, 1) dtype=float32, numpy=
 array([[0.8252393],
        [0.5630175]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [81]:
model.submodules

(<tensorflow.python.keras.engine.input_layer.InputLayer at 0x7ff8f0064080>,
 <tensorflow.python.keras.layers.core.Dense at 0x7ff8f00640b8>)

In [82]:
model.layers

In [83]:
model.name

'sequential'

In [84]:
model.name_scope()

'sequential'